In [1]:
import os
import pandas as pd

# Advances in Machine Learning with Big Data

### Dr Jeremy Large and Dr Matthias Qian
#### jeremy.large@economics.ox.ac.uk
#### matthias.qian@economics.ox.ac.uk
### Trinity 2020



# Overview of the course

In [2]:
df = pd.read_excel(io="https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx", sheet_name="Year 2009-2010")

In [3]:
df.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,525461.0,10.337667,107.424110,-9600.00,1.00,3.0,10.00,19152.00
Price,525461.0,4.688834,146.126914,-53594.36,1.25,2.1,4.21,25111.09
Customer ID,417534.0,15360.645478,1680.811316,12346.00,13983.00,15311.0,16799.00,18287.00


### How could we make money from this data?

 * relate to prices on markets

 * figure out when/where/how/why products are being bought

 * anticipate government or other official statistics
 
 * ...

### What new possibilities does this data open up for us?

* Regressors -> numerous
* Datapoints -> very numerous
* (have to clean … a lot)
* Computers -> faster

### Techniques covered in this course:

Jeremy Large:
 * Resampling methods
   * Cross-validation
   * Bootstrap
 * Flexible regression
   * Non-linear regression
     * Generalized additive model
     * Projection pursuit
     * Single layer Neural Net Perceptron
   * Step functions in regression
     * Step function
     * Decision tree
     * Bagging
     * Random forest
     * Boosting

Matthias Qian:
 * Neural Net
 * Recursive Neural Net
 * ... tbc ...

### Important techniques not covered in this course:

 * Predictor selection (Ridge regression / LASSO(Lars) / PCR / Resampling & brute force)
 
 * Classification

### Comments:

 * Burning Glass will be our dataset throughout this course
 
   * we will provide access to the data as follows ...